# CO2 and Weather

## Formalia:

The analysis in this notebook, will explore the CO2 on a Worldwide and EU scale. Where a deeper look will be taken on the top 5 with respectively highest and lowest CO2 emissions. A comparison between EU and those countries will also be carried out. 

## Part 1: Development of CO2 emission in the world

In [ ]:
# List of EU27 countries (excluding UK)
EU = ['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden'] 

#### Preparing the data to fit the format needed for the analysis


In [ ]:
# Import libraries need for the analysis
from math import ceil
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import geojson
#Only two digit after the decimal point
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning

In [ ]:
# Cleaning the data and renaming the columns
df= pd.read_csv('annual-co2-emissions-per-country.csv')
#replace NaN with 0
df = df.fillna(0)  
#replace inf with 0
df = df.replace([np.inf, -np.inf], 0)

# if Slovak Republic then replace with Slovakia
df.replace(to_replace='Slovak Republic', value='Slovakia', inplace=True)
# Czech Republic to Czechia
df.replace(to_replace='Czech Republic', value = 'Czechia', inplace=True)
df = df.drop(columns=['Code'])
df = df.rename(columns={'Entity': 'Country Name'})

# How does it look
df.head()

In [ ]:
# Change the data to be in the format of a table
df = df.pivot(index='Country Name', columns='Year', values='Annual CO2 emissions')
df = df.reset_index()
df = df.rename(columns={'Country Name': 'Country Name'})
df = df.set_index('Country Name')
df = df.fillna(0)
# remove the word Year, so Country moved one up
df.columns.name = None
df = df.reset_index()

In [ ]:
# Removing the years that are not needed - only looking at 1960 onwards
col_list = []
for year in range(1750, 1960):
    col_list.append(int(year))

# drop column thats in the col_list
df.drop(columns=col_list, axis=1, inplace=True)
df.head()

In [ ]:
# Keep the original data safe and create a new dataframe for the EU countries
df_EU = df.copy()
df_EU = df_EU[df_EU['Country Name'].isin(EU)]

In [ ]:
# Some basis statistics using stats for EU CO2 emission
df_EU.describe()


In [ ]:
df_EU_yearly = df.copy()
df_EU_yearly = df_EU.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')
df_EU_yearly.head()


#### Visualization of the data

In [ ]:
# Creates a bar chart for each country in the EU using supblot to show each country in a different plot
col = 3
rows = math.ceil(len(EU) / col)


fig, ax = plt.subplots(rows, col, sharex=True, figsize=(24, 15))

for index, country in enumerate(EU):
    plt.subplot(rows, col, index+1)
    plt.title(country)#, x = 0.1, y=0.7, pad=20) 
    plt.xlabel('Year')
    plt.ylabel('CO2 Emission')
    y_min, y_max = 0, df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'].max()   
    #print(f'{country} --> y_min: {y_min}, y_max: {y_max}')
    plt.ylim([y_min, y_max])
    y_interval = int((y_max - y_min) / 5) if y_max > 5 else 5
    plt.yticks(range(0, int(y_max), y_interval))
    plt.xticks(rotation=90)
    # Bar chart for each country with the CO2 emission for each year
    plt.bar(df_EU_yearly[df_EU_yearly['Country Name'] == country]['Year'], df_EU_yearly[df_EU_yearly['Country Name'] == country]['CO2 Emission'])
   
    plt.tight_layout()
    plt.title('CO2 Emission for ' + country)
plt.tight_layout()

# save to html
plt.savefig('CO2_Emission_EU.png')
fig.suptitle('CO2 Emission for each country in the EU distribuated through the years 1960-2018', fontsize=16, y=1.05)
plt.show()

The plot above shows the CO2 development for each country in the EU, this plot illustrate which countries has the higest and lowest CO2 emission. 
The y-axis represents the total CO2 emission, while the x-axis represents the years starting from 1960 til 2022. 
It can be seen that throughout the years, Germany have the highest emissions, while a country like Malta has the lowest emissions.

In [ ]:
# Total emission for world for each year
df_world = df.copy()
# should only have the Country World
df_world = df_world[df_world['Country Name'] == 'World']
# drop the Country World
df_world = df_world.sum(axis=0)
df_world = df_world.reset_index()
df_world.columns = ['Year', 'CO2 Emission']
#remove max_year and year_max
df_world = df_world.drop(df_world[df_world['Year'] == 'max_year'].index)
df_world = df_world.drop(df_world[df_world['Year'] == 'year_max'].index)
#plot the total CO2 emission for the world
fig = px.line(df_world, x='Year', y='CO2 Emission',title='CO2 Emission per year for the world from 1960-2022')
fig.show()
fig.write_html("CO2_Emission_world_1960_2022.html")

This plot shows the total CO2 emission for the world from 1960-2022. 
The y-axis represents total CO2 emission, while the x-axis represents the years.

The plot shows that the CO2 emission has been increasing over the years.
Worth noting is 1973-1975, 1979-1980, 1991-1992, 2009 and 2020. Seems like something happen in those years, since the curve breaks in those places.

The choice of a line plot is motivated by wanting to show the worlds effort to reduce CO2 emissions over time. By visually tracking this journey, the line plot will show insights into which direction the world is moving towards, to limit CO2 emissions. It also shows the different events that might have happened. 

In [ ]:
# Timeserie with world co2 emission
df_world = df.copy()

list_world = ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 
              'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico', 'Micronesia', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Korea', 'North Macedonia', 'Norway', 'Oman', 'Pakistan',
              'Palau', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Korea', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican City', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe']

df_world = df_world[df_world['Country Name'].isin(list_world)]
df_world = df_world[~df_world['Country Name'].isin(EU)]
df_world_yearly = df_world.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')
#plot the world co2 emission
fig = px.line(df_world_yearly, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for each country in the world except EU countries')
fig.show()
# save interactive figure as html
fig.write_html("CO2_Emission_World.html")

This plot shows every country in the world except for the EU countries. The y-axis represents the total amount of CO2, while the x-axis represents the years. 

It can clearly be seen the top 5 emitters in the world, but the top 5 lowest is harder.
To see the top 5 lowest emitters, it needs to be zoomed all the way in.

Top 5 highest = ['China', 'United States', 'India', 'Russia', 'Japan']

Top 5 lowest =  ['Tuvalu', 'Nauru', 'Kiribati', 'Sao Tome and Principe', 'Liechtenstein']

By using the line-plot for each country, we get an indication of the each country's progress towards limiting CO2 emissions. By adding all the countries in the plot, we can compare and see which country that has the highest CO2 emission compared to the other countries.

In [ ]:
# Top 10 list with highest CO2 emission in 2018
top_5 = ['China', 'United States', 'India', 'Russia', 'Japan']
highest_country = df.copy()
highest_country =  highest_country[highest_country['Country Name'].isin(top_5)]
highest_country =highest_country.reset_index()
highest_country = highest_country.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')


df_EU_total = df_EU.copy()
df_EU_total = df_EU_total.drop(columns=['Country Name'])
df_EU_total = df_EU_total.sum(axis=0)
df_EU_total = df_EU_total.reset_index()
df_EU_total.columns = ['Year', 'CO2 Emission']
df_EU_total['Country Name'] = 'EU'

# Combine the data for the EU and the top 5 countries
df_combine = pd.concat([df_EU_total, highest_country])

#remove column with NaN
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'max_year'].index)
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'year_max'].index)
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'year_min'].index)

#plot the total CO2 emission for the world
fig = px.line(df_combine, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for the current top 5 emitters compared to EU from 1960-2018')
fig.show()
fig.write_html("CO2_top_5_vs-EU_emission_world_1960_2018.html")

This is a line plot of the current 5 emitters compared to EU from 1960-2018.
The y-axis represents the total amount of CO2 emission, while the x-axis represents the years from 1960-2018. 
This plot give a good visualization on how EU compares to the top 5 emitters in the world.
It can be seen that while China and India is increasing, the others countries is either stable or lowering it's CO2.

In [ ]:
# Top 10 list with highest CO2 emission in 2018
lowest_top_5 = ['Tuvalu', 'Nauru', 'Kiribati', 'Sao Tome and Principe', 'Liechtenstein']
lowest_country = df.copy()
lowest_country =  lowest_country[lowest_country['Country Name'].isin(lowest_top_5)]
lowest_country =lowest_country.reset_index()
lowest_country = lowest_country.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')

# Combine the data for the EU and the top 5 countries
df_combine = pd.concat([df_EU_total, lowest_country])

#remove column with NaN
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'max_year'].index)
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'year_max'].index)
df_combine = df_combine.drop(df_combine[df_combine['Year'] == 'year_min'].index)

#plot the total CO2 emission for the world
fig = px.line(df_combine, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for the current lowest top 5 emitters compared to EU from 1960-2018')
fig.show()
fig.write_html("CO2_lowest_top_5_vs-EU_emission_world_1960_2018.html")

The plot is a line plot that showcases the CO2 emission per year for the current lowest top 5 emtters compared to EU from 1960-2018. The y-axis represents the total CO2 emission, while the x-axis represents the years from 1960-2018. 
This plot clearly showcase, how much more EU emits compared to those in the lowest end. When EU is shown, the others isn't shown - EU have to be removed before the development of CO2 emission can be seen for those countries with the lowest CO" emissions.

In [ ]:
# First first row is the year and first coloumn is the country name, make a plot where the x-axis is the year and the y-axis is the CO2 emission and make it so theres a subplot for each country
fig = px.line(df_EU_yearly, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per year for each country in the EU')
fig.show()
# save interactive figure as html
fig.write_html("CO2_Emission_EU.html")


This plot shows that Germany is clearly the highest emitter of CO2 throughout the years in EU. Again, need to zoom in to see the country with the lowest emissions.

In [ ]:
# plotly bar chart with country name, max_year, year_max, min_year and year_min
fig = go.Figure(data=[ go.Bar(name='Min CO2 Emission', x=df_EU['Country Name'], y=df_EU['min_year'], text=df_EU['year_min'], textposition='auto'),
                      go.Bar(name='Max CO2 Emission', x=df_EU['Country Name'], y=df_EU['max_year'], text=df_EU['year_max'], textposition='auto')])
fig.update_layout(title='Max and Min CO2 Emission and Year of Max and Min CO2 Emission for EU countries', barmode='group')
fig.write_html("Bar_chart_Max_and_Min_CO2_Emission_and_Year_of_Max_and_Min_CO2_Emission_for_EU_countries.html")
fig.show()

A bar chart that shows when a country has it lowest and highest CO2 emission. Again Germany tops, while Matla is in the lowest end of the spectra.

In [ ]:
# max value in df_EU_yearly
max_value = df_EU_yearly['CO2 Emission'].max()
# min value in df_EU_yearly
min_value = df_EU_yearly['CO2 Emission'].min()

In [ ]:
with  open('europe.geojson') as response: #### Using the geojson of lat and lon of San Fransisco
    counties = geojson.load(response) 

fig = px.choropleth(df_EU_yearly, 
                           geojson=counties, 
                           locations='Country Name', 
                           featureidkey='properties.NAME',
                           color='CO2 Emission',
                           color_continuous_scale= "turbo", ## ????
                           hover_name = 'Country Name',
                           hover_data=  ['Year', 'CO2 Emission'],
                           animation_frame='Year',
                           #locationmode='geojson-id',
                           range_color=(min_value, max_value),
                           #mapbox_style="carto-positron",
                           scope='europe',
                           center = {"lat":52.52000660, "lon": 13.404954007},
                           #opacity=1,
                           #labels={'CO2 Emission':'CO2 Emission'}
                           width=1000, height=800,
                           title = 'CO2 for EU from 1960-2018'
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#save as html
fig.write_html("CO2_Emission_EU_Map_Online.html", include_plotlyjs='cdn')
"""'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']."""

A great visualization of the development of CO2 for EU.

In [ ]:
# Cleaning the data and renaming the columns for the dataset used to CO2 per capita
df_per_capita= pd.read_csv('CO2_Emissions_1960-2018.xls')

#replace NaN with 0
df_per_capita = df_per_capita.fillna(0)  
#replace inf with 0
df_per_capita = df_per_capita.replace([np.inf, -np.inf], 0)

# if Slovak Republic then replace with Slovakia
df_per_capita.replace(to_replace='Slovak Republic', value='Slovakia', inplace=True)
# Czech Republic to Czechia
df_per_capita.replace(to_replace='Czech Republic', value = 'Czechia', inplace=True)

df_per_capita = df_per_capita.rename(columns={'Entity': 'Country Name'})

# How does it look
df_per_capita.head()

In [ ]:
top_2 = ['Germany', 'Malta']
co2_captia = df.copy()
co2_captia =  co2_captia[co2_captia['Country Name'].isin(top_2)]

# Time series with CO2 per capita for Germany and Malta
co2_captia = co2_captia.melt(id_vars='Country Name', var_name='Year', value_name='CO2 Emission')
fig = px.line(co2_captia, x='Year', y='CO2 Emission', color='Country Name', title='CO2 Emission per Capita for Germany and Malta (1960-2022)')
#use log scale
fig.update_yaxes(type="log")
fig.show()
# Save the file
fig.write_html("CO2_Emission_per_capita_Germany_Malta.html")


A time serie plot showing that when measured with CO2 per captia, Germany is still in the high end, while Malta is still the lowest.

## Part 2: 
## Exploring CO2 Emission in Europe: A look into the history of Highs and Lows
The following is focusing on visiualisations that shows the highs and lows of the co2 emission history in Europe.
Line plots, and visualization of geodata using plotly.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, find_peaks, find_peaks, find_peaks

# Only two digits after the decimal point
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Filter data for the years between 1960 and 2022
df_EU_yearly = df_EU_yearly[(df_EU_yearly['Year'] >= 1960) & (df_EU_yearly['Year'] <= 2022)]

# Total CO2 emissions for each year
total_CO2_each_year = df_EU_yearly.groupby('Year')['CO2 Emission'].sum()

# Find peaks in the data
peaks, _ = find_peaks(total_CO2_each_year.values)
valleys, _ = find_peaks(-total_CO2_each_year.values)

# Plot the total CO2 emission for each year
plt.figure(figsize=(15, 6))
plt.plot(total_CO2_each_year.index, total_CO2_each_year.values, linestyle='-')

# Plot peaks and valleys
plt.plot(total_CO2_each_year.index[peaks], total_CO2_each_year.values[peaks], marker='o', linestyle='', color='red', markersize=8)
plt.plot(total_CO2_each_year.index[valleys], total_CO2_each_year.values[valleys], marker='o', linestyle='', color='green', markersize=8)


plt.xlabel('Year')
plt.ylabel('Total CO2 Emission')
plt.title('Total CO2 Emission for each year (EU Countries)')
plt.xticks(total_CO2_each_year.index, rotation=90) 
plt.xlim(1960, 2022)
plt.grid()
plt.savefig('Total_CO2_Emission_for_each_year_EU.png')
plt.show()


This is a line plot that showcases the total amount of CO2 emission in europe each year. 
Y-axis shows the total amount of CO2, while the x-axis shows the years from 1960-2022.
The red dots showcases the peaks, while the green dots showcases the valleys. 
We do see a high increase of CO2 emission in year 1979, while the lowest CO2 emission after 1979 was in year 2020. In 1979 the oil crisis happened, while in 2020 the pandemic COVID 19 happened. 

The reasoning behind choosing to show the data through a line plot is that it effectively displays trends and patterns over a continuous range of time. Furthermore the line plot allows for easy comparison between the different years and makes it easier to identify overall trends, such as increasing or decreasing emissions over time. 

In [ ]:
# Filter data for the year 1979
df_1979 = df_EU_yearly[df_EU_yearly['Year'] == 1979]

# max value in df_1979
max_value = df_1979['CO2 Emission'].max()

# min value in df_1979
min_value = df_1979['CO2 Emission'].min()

with  open('europe.geojson') as response:
    counties = geojson.load(response) 

fig = px.choropleth(df_1979, 
                           geojson=counties, 
                           locations='Country Name', 
                           featureidkey='properties.NAME',
                           color='CO2 Emission',
                           color_continuous_scale= "turbo", 
                           hover_name = 'Country Name',
                           hover_data=  ['Year', 'CO2 Emission'],
                           animation_frame='Year',
                           range_color=(min_value, max_value),
                           scope='europe',
                           center = {"lat":52.52000660, "lon": 13.404954007},
                           width=700, height=500,
                           title = 'CO2 for EU from 1960-2018'
                          )
fig.update_layout(
    margin={"r":0, "t":0, "l":0, "b":0})
fig.show()

#save as html
fig.write_html("CO2_Emission_EU_Map_Online_1979.html", include_plotlyjs='cdn')
"""'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']."""

The colors on the map showcases the CO2 emission in EU in 1979. The reasoning behind showcasing CO2 emission for that year is because it is the year that EU experienced the most CO2 emission as seen in the Line plot. The closer the country is to the color red, the higher CO2 emission the country had that year, while the closer the country's color is to blue the less CO2 emission the country had that year. When zooming in, we do see that Germany is the only red country on the map, which means that Germany is the country in 1979 with the highest CO2 emission. Furthermore we see multiple countries that are dark blue on the map, where Malta is the country with the least CO2 emission that year. To make sure that these observations are correct, we just cross checked by doing the following. 

The reasoning behind choosing to show the data through a map plot, is to compare different regions. Map plots allows color-coding countries based on their CO2 emissions, which makes it easier to compare between the countries and see which countries had the highest or lowest CO2 emissions that specific year. Furthermore the ability to hover over specific countries, allows insights into country-level information.

In [ ]:
#Highest CO2 emission in 1979
highest_emission_row = df_1979.loc[df_1979['CO2 Emission'].idxmax()]

# Extract the country name and its corresponding CO2 emission value
highest_emission_country = highest_emission_row['Country Name']
highest_emission_value = highest_emission_row['CO2 Emission']

# Lowest CO2 emission in 1979
lowest_emission_row = df_1979.loc[df_1979['CO2 Emission'].idxmin()]

# Extract the country name and its corresponding CO2 emission value
lowest_emission_country = lowest_emission_row['Country Name']
lowest_emission_value = lowest_emission_row['CO2 Emission']

print("Country with the highest CO2 emission in 1979:", highest_emission_country)
print("CO2 emission value:", highest_emission_value)

print("\nCountry with the lowest CO2 emission in 1979:", lowest_emission_country)
print("CO2 emission value:", lowest_emission_value)


As seen in the print out statements, Germany is indeed the country that had the highest CO2 emission in year 1979, while Malta was the country with the lowest CO2 emission. Another interesting perspective was to look into 2020 and see which country had the highest CO2 emission and the country with the lowest CO2 emission.

The reasoning behind cross-checking is to verify the accuracy of the observations made from the map plot. While the map plot provides a visual representation of the data, it is not always easy to see which country has the highest or the lowest CO2 emissions. If the color-coding of multiple countries are the same or if a country is very small, it can be hard to classify which country has the highest or lowest CO2 emissions. By using code to extract highest and lowest country, we can ensure that the observations are based on accurate data rather than subjective interpretation.

In [ ]:
# Filter data for the year 2020
df_2020 = df_EU_yearly[df_EU_yearly['Year'] == 2020]

# max value in df_2020
max_value = df_2020['CO2 Emission'].max()

# min value in df_2020
min_value = df_2020['CO2 Emission'].min()

with  open('europe.geojson') as response:
    counties = geojson.load(response) 

fig = px.choropleth(df_2020, 
                           geojson=counties, 
                           locations='Country Name', 
                           featureidkey='properties.NAME',
                           color='CO2 Emission',
                           color_continuous_scale= "turbo", 
                           hover_name = 'Country Name',
                           hover_data=  ['Year', 'CO2 Emission'],
                           animation_frame='Year',
                           range_color=(min_value, max_value),
                           scope='europe',
                           center = {"lat":52.52000660, "lon": 13.404954007},
                           width=700, height=500,
                           title = 'CO2 for EU from 1960-2018'
                          )
fig.update_layout(
    margin={"r":0, "t":0, "l":0, "b":0})
fig.show()
#save as html
fig.write_html("CO2_Emission_EU_Map_Online_2020.html", include_plotlyjs='cdn')
"""'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']."""

The colors on the map showcases the CO2 emission in EU in 2020. This was the year where EU experienced the least CO2 emission after 1979 as seen in the Line plot. 
The closer the country is to the color red, the higher CO2 emission the country had that year, while the closer the country's color is to blue the less CO2 emission the country had that year. When zooming in, we do see that Germany is once again the only country that is red on the map, which means that Germany is the country in 2020 with the highest CO2 emission. Furthermore we see multiple countries that are dark blue on the map, where Malta is the country with the least CO2 emission that year. To make sure that the observations are correct, we once again cross checked by doing the following. 

The reasoning behind showing the map plot in 2020, is the same reason as showing the map plot of EU in 1979. It makes it easier to compare between the countries and see which countries had the highest and lowest CO2 emissions in that specific year based on color-code. 

In [ ]:
#Highest CO2 emission in 2020
highest_emission_row = df_2020.loc[df_2020['CO2 Emission'].idxmax()]

# Extract the country name and its corresponding CO2 emission value
highest_emission_country = highest_emission_row['Country Name']
highest_emission_value = highest_emission_row['CO2 Emission']

# Lowest CO2 emission in 2020
lowest_emission_row = df_2020.loc[df_2020['CO2 Emission'].idxmin()]

# Extract the country name and its corresponding CO2 emission value
lowest_emission_country = lowest_emission_row['Country Name']
lowest_emission_value = lowest_emission_row['CO2 Emission']

print("Country with the highest CO2 emission in 2020:", highest_emission_country)
print("CO2 emission value:", highest_emission_value)

print("\nCountry with the lowest CO2 emission in 2020:", lowest_emission_country)
print("CO2 emission value:", lowest_emission_value)


As mentioned before, Germany and Malta keeps their spot as Germany is the country with highest CO2 emission and Malta is the country with lowest CO2 emission.

The reasoning behind cross-checking, is the same as before. To ensure that the observations are based on accurate data rather than subjective interpretation.

In [ ]:
# Filter data for Germany
df_Germany = df_EU_yearly[df_EU_yearly['Country Name'] == 'Germany']

# Total CO2 emissions for each year in Germany
total_CO2_each_year_Germany = df_Germany.groupby('Year')['CO2 Emission'].sum()

# Plot the total CO2 emission for each year in Germany
plt.figure(figsize=(15, 6))
plt.plot(total_CO2_each_year_Germany.index, total_CO2_each_year_Germany.values, linestyle='-')

plt.xlabel('Year')
plt.ylabel('Total CO2 Emission')
plt.title('Total CO2 Emission for each year (Germany)')
plt.xticks(total_CO2_each_year_Germany.index, rotation=90) 
plt.xlim(1960, 2022)
plt.grid()
plt.savefig('Total_CO2_Emission_for_each_year_Germany.png')
plt.show()


This is a line plot represents the total CO2 emission for each year in Germany from 1960-2022. The y-axis showcases the total CO2 emission of Germany, while the x-axis represents the years. 
As seen in the plots, Germany has the highest CO2 emission in 1979, while the lowest CO2 emission in 2020. We also see a overall decrease in CO2 emission after 1979.

The reasoning behind showing the data through a line plot, is the same reason as showing EU CO2 emission from 1960-1979. It overall allows us to see Germany's CO2 emissions journey and allow us to see which events affected the CO2 emissions the most. It also shows us if Germany is moving towards the right the direction, when it comes to minimizing CO2 emissions.

## Part 3: Extreme weather 

In extreme weather we want to investigate the correlation between natural disasters and CO2 emission. In order to do that we use two datasets, one for the Europe and World wide. 
There has been chosen interactive, bokeh,  plots for all the plots in order for the reader so play around with the data of the plot and get more details. The bokeh plots will provide the reader with support to investigate the values in the plot and as seen in some of the plots, the user can select specific natural disasters to see.

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_file
from bokeh.models import Legend, HoverTool, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import FixedTicker

#Importing and defining global variables to use all around
# For EU dataset 
excel_file_EU = 'EU_disasters.xlsx'

df = pd.read_excel(excel_file_EU)

df.drop(columns=['Historic', 'Classification Key','External IDs', 'Event Name','OFDA/BHA Response','Appeal','Declaration',"AID Contribution ('000 US$)", 'Magnitude','Magnitude Scale', 'Latitude','Longitude','River Basin','Start Day','Total Deaths','No. Injured','No. Affected','No. Homeless','Total Affected',"Reconstruction Costs ('000 US$)", 
                 "Reconstruction Costs, Adjusted ('000 US$)","Insured Damage ('000 US$)", "Insured Damage, Adjusted ('000 US$)", "Total Damage ('000 US$)","Total Damage, Adjusted ('000 US$)",'CPI', "Admin Units","Entry Date", 'Last Update','Origin','Associated Types'], inplace=True)

# Convert start year to datetime format
df['Start Year'] = pd.to_datetime(df['Start Year'], format='%Y').dt.year

# For WorldWide dataset
excel_file_World = 'WW.xlsx'

df_world = pd.read_excel(excel_file_World)
df_world.drop(columns=['Historic', 'Classification Key','External IDs', 'Event Name','OFDA/BHA Response','Appeal','Declaration',"AID Contribution ('000 US$)", 'Magnitude','Magnitude Scale', 'Latitude','Longitude','River Basin','Start Day','Total Deaths','No. Injured','No. Affected','No. Homeless','Total Affected',"Reconstruction Costs ('000 US$)", 
                 "Reconstruction Costs, Adjusted ('000 US$)","Insured Damage ('000 US$)", "Insured Damage, Adjusted ('000 US$)", "Total Damage ('000 US$)","Total Damage, Adjusted ('000 US$)",'CPI', "Admin Units","Entry Date", 'Last Update','Origin','Associated Types'], inplace=True)


df_world['Start Year'] = pd.to_datetime(df['Start Year'], format='%Y').dt.year



# global values
generic_colors = ['red', 'yellow', 'purple', 'blue', 'green', 'black', 'gray', 'pink', 'brown',
                  'skyblue', 'darkorange', 'lightseagreen', 'cyan', 'magenta', 'lime', 'gold',
                  'indigo', 'teal', 'salmon', 'peru', 'olive', 'orchid', 'steelblue','black']

disaster_colors = {
    "Flood": "blue",
    "Storm": "red",
    "Earthquake": "green",
    "Epidemic": "orange",
    "Glacial lake outburst flood": "purple",
    "Extreme temperature" :"magenta",
    "Mass movement (wet)":"lime",
    "Volcanic activity":"indigo",
    "Wildfire":"steelblue",
    "Drought":"peru",
    "Infestation":"olive",
    "Mass movement (dry)":"magenta",
}


In [4]:
# Print the lenght of the dataset of Europe
print(len(df))
df.head()

1824


,DisNo.,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,ISO,Country,Subregion,Region,Location,Start Year,Start Month,End Year,End Month,End Day
0,2000-0024-NOR,Technological,Transport,Rail,Rail,NOR,Norway,Northern Europe,Europe,Aasta (Hedmark),2000,1.0,2000,1.0,4.0
1,2000-0028-ROU,Natural,Meteorological,Storm,Blizzard/Winter storm,ROU,Romania,Eastern Europe,Europe,"Bihor, Satu Mare, Cluj, Salaj, Mures provinces",2000,1.0,2000,1.0,NaN
2,2000-0029-BGR,Natural,Meteorological,Extreme temperature,Cold wave,BGR,Bulgaria,Eastern Europe,Europe,"Sofia province, Sofia-city province",2000,1.0,2000,1.0,NaN
3,2000-0032-CHE,Technological,Transport,Air,Air,CHE,Switzerland,Western Europe,Europe,Niederhasli (Zurich),2000,1.0,2000,1.0,10.0
4,2000-0039-ESP,Technological,Miscellaneous accident,Collapse (Miscellaneous),Collapse (Miscellaneous),ESP,Spain,Southern Europe,Europe,Dos Hermanas (Séville),2000,2.0,2000,2.0,3.0


In [5]:
# Print the lenght of the dataset of World wide
print(len(df_world))
df_world.head()

15718


,DisNo.,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,ISO,Country,Subregion,Region,Location,Start Year,Start Month,End Year,End Month,End Day
0,1999-9388-DJI,Natural,Climatological,Drought,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,"Ali Sabieh, Dikhil, Djibouti, Obock, Tadjourah...",2000.0,6.0,2001,NaN,NaN
1,1999-9388-SDN,Natural,Climatological,Drought,Drought,SDN,Sudan,Northern Africa,Africa,"Northern Darfur, Northern Kordofan, Red Sea pr...",2000.0,1.0,2001,NaN,NaN
2,1999-9388-SOM,Natural,Climatological,Drought,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,"Ceel Barde, Rab Dhuure, Tayeeglow, Xudur distr...",2000.0,1.0,2001,NaN,NaN
3,2000-0001-AGO,Technological,Transport,Road,Road,AGO,Angola,Sub-Saharan Africa,Africa,Calulo,2000.0,1.0,2000,1.0,26.0
4,2000-0002-AGO,Natural,Hydrological,Flood,Riverine flood,AGO,Angola,Sub-Saharan Africa,Africa,"Dombre Grande village (Baia Farta district, Be...",2000.0,1.0,2000,1.0,15.0


In [11]:
###### Filter the two dataset to only consider the natural disasters 

# Filter the EU dataset 
filter_natural = df[df['Disaster Group'] == 'Natural']
filter_natural = filter_natural[~filter_natural['Country'].isin(['Russian Federation', 'United Kingdom of Great Britain and Northern Ireland'])]
print(len(filter_natural))
grouped_natural_disaster = filter_natural.groupby('Disaster Group')

# Filter the WW dataset 
filter_natural_world = df_world[df_world['Disaster Group'] == 'Natural']
print(len(filter_natural_world))


1108
9993


Let us look into the sum of all disaster each year in EU to see if we can see a correlation between previous blog plot of CO2 emission in EU. We only consider 'Start year' as we are interested in what year they started versus how long they lasted. The reason for this is to compare it to CO2 emission, we are looking anually and want to see of the anually CO2 emission affect the anually natural disasters. <br> The next plot is an a interactive line plot making it easy for reader to get an overview of the natural disasters. A hoverover tool is enabled making it easy for the reader to investigate each year's number of natural disasters.

In [9]:
# Plot for the sum of all disasters each year in EU 

grouped_start_year = filter_natural.groupby('Start Year').size()

p = figure(title="Number of Natural Disasters in EU in each year", x_axis_label='Start Year', y_axis_label='Number of Natural Disasters in EU', width=800, height=400)

p.line(grouped_start_year.index, grouped_start_year.values, line_width=2)

p.xaxis.ticker = FixedTicker(ticks=grouped_start_year.index.tolist())
p.xaxis.major_label_orientation = np.pi / 4

hover = HoverTool()
hover.tooltips = [("Year", "@x"), ("Occurrences", "@y")]
p.add_tools(hover)

output_file("natural_disasters_all_year_plot_EU.html")

show(p)


25


Let us go deeper into detail and include all the disaster types in each year as well. The next plot is a interactive bar chart instead of alternatives, as it gives a clear visualization of the extend each go to. Each bar has a hoover over tool enabled as none of the bars overlap and it makes it easy for the reader to interpret the information.

In [5]:
# Plot occurences of all disaster type for EU and WW

# Defining the method

def plot_diaster_types(dataset, colors, string, title):
    p = figure(x_range=dataset.index.astype(str).tolist(), height=400, width=700, title=title,
            toolbar_location=None, tools="")

    disaster_types = dataset.columns.tolist()
    bar_width = 1 / len(disaster_types)  
    renderers = [] 

    for i, disaster_type in enumerate(disaster_types):
        x_pos = [year + i * bar_width for year in range(len(dataset))]
        source = ColumnDataSource(data=dict(x=x_pos, top=dataset[disaster_type], name=[disaster_type] * len(x_pos), year=dataset.index.astype(str).tolist()))
        renderer = p.vbar(x='x', top='top', width=bar_width, color=colors[disaster_type], source=source)
        renderers.append(renderer)  
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Number of Disasters'
    p.xaxis.major_label_orientation = np.pi / 4

    hover = HoverTool(tooltips=[("Year", "@year"), ("Disaster Type", "@name"), ("Occurrence", "@top")], renderers=renderers)
    p.add_tools(hover)

    legend_items = [(disaster_type, [renderer]) for disaster_type, renderer in zip(disaster_types, renderers)]
    legend = Legend(items=legend_items, location="top_right", click_policy='hide')
    p.add_layout(legend, 'right')

    output_file(string + ".html")
    show(p)


# Plot  for EU
grouped_year = filter_natural.groupby(['Start Year', 'Disaster Type']).size().unstack(fill_value=0)
plot_diaster_types(grouped_year, disaster_colors, "plot_disaster_EU_all_years", "Number of Natural Disasters by Type and Year in EU")


In the next section, we will look into the countries producing the most and least amount of CO2 in Europe. World wide we will look into the countries with the highest emission. These next plots are interactive bar plots where in the Legend the disaster types can be hidden with a click. No hoover over tool is enabled, as we see that the natural disasters can be stacked on each meaning there can be several occurences in the same year with the same number making it hard to interpret the hoover over tool as they too will be stacked/overlap each other. <br>
<br>For EU we see that Germany and Italy are the countries with the highest CO2 emission, while Malta and Latvia are the countries with lowest. <br>
Worldwide China and USA are the two countries having the highest CO2 emission. 


In [10]:
###### Plot for selected countries
grouped_country = filter_natural.groupby(['Country', 'Start Year', 'Disaster Type']).size().unstack(fill_value=0)
grouped_country_WW = filter_natural_world.groupby(['Country', 'Start Year', 'Disaster Type']).size().unstack(fill_value=0)


def disastertypes_country(dataset, countries, colors, name_of_file):
    plots = []

    num_countries = len(countries)
    plots_per_row = 2
    num_rows = (num_countries + plots_per_row - 1) // plots_per_row

    disaster_names = []

    for idx, country in enumerate(countries):
        country_data = dataset.loc[country]
        disaster_types = country_data.columns
        
        all_years = np.arange(country_data.index.min(), country_data.index.max() + 1)

        non_zero_disasters = [disaster_type for disaster_type in disaster_types if country_data[disaster_type].sum() > 0]

        p = figure(title=f'Number of Natural Disasters by Type and Year in {country}',
                    x_axis_label='Year', y_axis_label='Number of Disasters',
                    x_range=(min(all_years)-1, max(all_years)+1), width=800, height=400)

        plotted_disaster_types = []

        for i, disaster_type in enumerate(non_zero_disasters):
            x = all_years
            y = country_data[disaster_type].reindex(all_years, fill_value=0)
            source = ColumnDataSource(data=dict(x=x, y=y, name=[disaster_type] * len(x)))
            bars = p.vbar(x='x', top='y', width=0.8, color=colors[disaster_type], alpha=0.5, source=source)
            plotted_disaster_types.append((disaster_type, [bars]))
            disaster_names.append(disaster_type)  

        p.xaxis.ticker = all_years
        p.xaxis.major_label_orientation = np.pi / 4
        p.yaxis.ticker = np.arange(0, 6, 1)
        legend = Legend(items=plotted_disaster_types, location="top_left", click_policy="hide")
        legend.spacing = 5
        p.add_layout(legend, 'right')
        plots.append(p)

    grid = gridplot(plots, ncols=plots_per_row)

    output_file(name_of_file + ".html")
    show(grid)



##################### PLOTS FOR EU ####################

#1. Highest Germany
countries_to_include_estonia = ['Germany']
disastertypes_country(grouped_country, countries_to_include_estonia, disaster_colors, "Germany_EU")


#2. Highest Italy 
countries_to_include_sweden = ['Italy']
disastertypes_country(grouped_country, countries_to_include_sweden, disaster_colors, "Italy_EU")


# 1. lowest Malta
countries_to_include_malta = ['Malta']
disastertypes_country(grouped_country, countries_to_include_malta, disaster_colors, "Malta_EU")

# 2. lowest Latvia
countries_to_include_malta = ['Latvia']
disastertypes_country(grouped_country, countries_to_include_malta, disaster_colors, "Latvia_EU")


##################### PLOTS FOR WW ####################

# 1. Highest WW
countries_to_include_WW_China = ['China']
disastertypes_country(grouped_country_WW, countries_to_include_WW_China, disaster_colors, "China_WW")


# 2. Highest WW
countries_to_include_WW_China = ['United States of America']
disastertypes_country(grouped_country_WW, countries_to_include_WW_China, disaster_colors, "USA_WW")


Index(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas',
       ...
       'United Republic of Tanzania', 'United States of America', 'Uruguay',
       'Uzbekistan', 'Vanuatu', 'Venezuela (Bolivarian Republic of)',
       'Viet Nam', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country', length=154)
